In [10]:
print("hello world")

hello world


In [11]:
import torch
import torch.nn as nn

# My code starts here

In [75]:
class ResidualBlock(nn.Module):
    def __init__(self, in_channels, out_channels, kernel_size=3):
        super(ResidualBlock,self).__init__()
        self.conv1 = nn.Conv2d(in_channels, out_channels, kernel_size, padding=1)
        self.conv2 = nn.Conv2d(out_channels, out_channels, kernel_size, padding=1)
        #batch norm layer
        self.bn1 = nn.BatchNorm2d(num_features=out_channels)
        self.bn2 = nn.BatchNorm2d(num_features=out_channels)
        self.relu = nn.ReLU(inplace=True)
        
        #transition the input channel for the residual path to add
        self.trans = nn.Conv2d(in_channels, out_channels, kernel_size=1)
        #if in_channels == out_channels: no need to do transition
        self.isChanged = not in_channels == out_channels
        
    def forward(self, x):
        print(x.shape)
        f_x = self.conv1(x)
        f_x = self.bn1(f_x)
        f_x = self.relu(f_x)
        f_x = self.conv2(f_x)
        f_x = self.bn2(f_x)
        
        #residual path
        print(f_x.shape)
        #in order for the residual path to work, we need to match the channels
        if(self.isChanged):
            x = self.trans(x)
        print(x.shape)
        x = f_x + x
        x = self.relu(x)
        return x

In [57]:
from torchsummary import summary

In [76]:
residualBlock = ResidualBlock(3,20)

In [77]:
summary(residualBlock, (3, 32, 32))

torch.Size([2, 3, 32, 32])
torch.Size([2, 20, 32, 32])
torch.Size([2, 20, 32, 32])
----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 20, 32, 32]             560
       BatchNorm2d-2           [-1, 20, 32, 32]              40
              ReLU-3           [-1, 20, 32, 32]               0
            Conv2d-4           [-1, 20, 32, 32]           3,620
       BatchNorm2d-5           [-1, 20, 32, 32]              40
            Conv2d-6           [-1, 20, 32, 32]              80
              ReLU-7           [-1, 20, 32, 32]               0
Total params: 4,340
Trainable params: 4,340
Non-trainable params: 0
----------------------------------------------------------------
Input size (MB): 0.01
Forward/backward pass size (MB): 1.09
Params size (MB): 0.02
Estimated Total Size (MB): 1.12
----------------------------------------------------------------
